## Procesamiento Analytics
### Objetivos
- Preparación de datos para uso en presentación y gráficos

In [2]:
import pandas as pd

df_hosts = pd.read_csv('datos/refined/olympic_hosts.csv')
df_results = pd.read_csv('datos/refined/olympic_results.csv')
df_medals = pd.read_csv('datos/refined/olympic_medals.csv')
df_atletas = pd.read_csv('datos/refined/olympic_athletes.csv')

### Medallas

In [4]:
# Crear DataFrame sin duplicados para los equipos
df_medals_equipos = df_medals[df_medals['participant_type'] == 'GameTeam'].drop_duplicates(subset=['participant_title', 'medal_type'])
df_medals_equipos.drop(columns=['athlete_full_name'],inplace=True)

# Crear DataFrame para participantes individuales
df_medals_individuales = df_medals[df_medals['participant_type'] == 'Athlete']

# Combinar los DataFrames de equipos y participantes individuales
# En este dataset no hay rows repetidas por cada miembro de un equipo
df_medals_combined = pd.concat([df_medals_equipos, df_medals_individuales])

# Contar las medallas por tipo, país, y disciplina
df_medals_by_country_discipline = df_medals_combined.groupby(['converted_country_name', 'discipline_title', 'medal_type']).size().reset_index(name='medal_count')
df_medals_by_country_discipline = df_medals_by_country_discipline.sort_values(by='medal_count', ascending=False)

# Mostrar el DataFrame con medallas por tipo, país, y disciplina
display(df_medals_by_country_discipline)

,converted_country_name,discipline_title,medal_type,medal_count
2722,United States,Athletics,GOLD,288
2723,United States,Athletics,SILVER,255
2721,United States,Athletics,BRONZE,209
2822,United States,Swimming,GOLD,187
2823,United States,Swimming,SILVER,162
...,...,...,...,...
779,Egypt,Judo,SILVER,1
780,Egypt,Karate,BRONZE,1
2116,Slovenia,Sailing,BRONZE,1
2115,Slovenia,Rowing,SILVER,1


In [5]:
# Filtrar df_medals para incluir solo las filas con athlete_full_name no nulos y no vacíos
df_medals_con_nombre = df_medals[df_medals['athlete_full_name'].notna() & (df_medals['athlete_full_name'] != '')]

# Merge con df_atletas en base a la columna 'athlete_full_name'
df_medals_atletas = pd.merge(df_medals_con_nombre, df_atletas, on='athlete_full_name', how='inner')

# display(df_medals_atletas)

# Realizar un merge en base a 'game_slug' para traer 'game_year' de df_hosts
df_medals_atletas_hosts = pd.merge(df_medals_atletas, df_hosts, on='game_slug', how='inner')

# Filtrar df_medals_atletas_hosts para eliminar las filas donde birth_year_estimated es True
df_medals_atletas_hosts = df_medals_atletas_hosts[df_medals_atletas['birth_year_estimated'] == False]

# Calcular la edad de los atletas cuando ganaron la medalla
df_medals_atletas_hosts['age_at_medal'] = df_medals_atletas_hosts['game_year'] - df_medals_atletas_hosts['athlete_year_birth']

# display(df_medals_atletas_hosts)

# Encontrar el atleta más joven en cada combinación de país, tipo de medalla y temporada
youngest_medalists = df_medals_atletas_hosts.loc[df_medals_atletas_hosts.groupby(['country_name', 'medal_type', 'game_season'])['age_at_medal'].idxmin()]

# Crear un nuevo DataFrame con las columnas deseadas
df_youngest_medalists = youngest_medalists[['country_name', 'medal_type', 'game_season', 'athlete_full_name', 'age_at_medal']].reset_index(drop=True)

display(df_youngest_medalists)

,country_name,medal_type,game_season,athlete_full_name,age_at_medal
0,Afghanistan,BRONZE,Summer,Rohullah NIKPAI,21.0
1,Algeria,BRONZE,Summer,Mohamed BAHARI,20.0
2,Algeria,GOLD,Summer,Taoufik MAKHLOUFI,24.0
3,Algeria,SILVER,Summer,Ali SAIDI-SIEF,22.0
4,Argentina,BRONZE,Summer,Francisco RISIGLIONE,19.0
...,...,...,...,...,...
475,Yugoslavia,SILVER,Winter,Mateja SVET,20.0
476,Zambia,SILVER,Summer,Samuel MATETE,28.0
477,Zimbabwe,BRONZE,Summer,Kirsty Leigh COVENTRY,21.0
478,Zimbabwe,GOLD,Summer,Kirsty Leigh COVENTRY,21.0


In [6]:
# Contar las medallas por tipo, país, y género
df_medals_by_country_gender = df_medals_combined.groupby(['converted_country_name', 'event_gender', 'medal_type']).size().reset_index(name='medal_count')
df_medals_by_country_gender = df_medals_by_country_gender.sort_values(by='medal_count', ascending=False)

### Resultados

In [8]:
# Merge Resultados, Medallas, y Hosts

# Realizar un merge en base a 'game_slug' para traer 'game_year' de df_hosts
df_merged_results_hosts = pd.merge(df_results, df_hosts, on='game_slug', how='inner')

# Realizar un merge en base a 'game_slug' y 'event_title' para traer 'event_gender' de df_medals
df_merged_results_hosts = pd.merge(df_merged_results_hosts, df_medals[['game_slug', 'event_title', 'event_gender']], on=['game_slug', 'event_title'], how='left')

# Agrupar participaciones por género, y año
df_participaciones_by_gender_year = df_merged_results_hosts.groupby(['event_gender', 'game_year']).size().reset_index(name='participation_count')

display(df_participaciones_by_gender_year)

,event_gender,game_year,participation_count
0,Men,1896,1388
1,Men,1900,2943
2,Men,1904,3926
3,Men,1908,5762
4,Men,1912,8278
...,...,...,...
138,Women,2014,5209
139,Women,2016,13791
140,Women,2018,5574
141,Women,2020,18967


In [9]:
# Eliminar duplicados para que cada país esté representado solo una vez por año y temporada
df_unique_countries = df_merged_results_hosts[['game_year', 'game_season', 'country_name']].drop_duplicates()

# Agrupar por año y temporada, y contar el número de países únicos
df_countries_per_year_season = df_unique_countries.groupby(['game_year', 'game_season'])['country_name'].nunique().reset_index()

# Renombrar la columna para mayor claridad
df_countries_per_year_season = df_countries_per_year_season.rename(columns={'country_name': 'unique_country_count'})

# Ordenar por año
df_countries_per_year_season = df_countries_per_year_season.sort_values(by='game_year', ascending=True)

display(df_countries_per_year_season)

,game_year,game_season,unique_country_count
0,1896,Summer,14
1,1900,Summer,28
2,1904,Summer,16
3,1908,Summer,24
4,1912,Summer,27
5,1920,Summer,30
6,1924,Summer,44
7,1924,Winter,17
8,1928,Summer,45
9,1928,Winter,25


### Guardamos los DataFrames generados en el área analytics

In [11]:
df_medals_by_country_discipline.to_csv('datos/analytics/olympic_medals_by_country_discipline.csv', index=False)
df_medals_by_country_gender.to_csv('datos/analytics/olympic_medals_by_country_gender.csv', index=False)
df_youngest_medalists.to_csv('datos/analytics/olympic_youngest_medalists.csv', index=False)

df_participaciones_by_gender_year.to_csv('datos/analytics/olympic_participaciones_by_gender_year.csv', index=False)
df_countries_per_year_season.to_csv('datos/analytics/olympic_countries_per_year_season.csv', index=False)